# Análisis del mercado cripto utilizando Python y Kafka.

### Objetivos del Proyecto:

A grandes rasgos, el objetivo es replicar un bot de trading, sin la necesidad de que se realicen operaciones en el exchange, y será algo sencillo, ya que no nos interesa tanto el caso de uso, si no que aprendas a manejar kafka con datos en tiempo real y a desacoplar servicios mediante su uso.
   
La idea es que obtengas datos en tiempo real del mercado, mediante un streaming
manejado en kafka, por lo que el proyecto estará dividido en 3 segmentos:
   
* Funciones Objetivo.
* Funciones Extras.
* Incorporar Servicios:
    - Visualización de Datos.
    - Conexión a Base de Datos (este lo dejamos en la primer sección).
    - Servicios de ML.
    - Data Analytics.
    - Reportes.

### Instrucciones

* Recuerda que lo primero es hacer el fork del repo y posterior a eso crear una carpeta personal en tu repositorio, de esta forma podrás hacer un push a la rama de python, con todo tu código dentro de tu carpeta, nada fuera de ella.

###### NO DEBES MODIFICAR NADA EN EL REPO ORIGINAL!!!

* Crear las funciones enunciadas en los archivos *producer_functions.py*, *consumer_functions.py* y *project_functions.py* para obtener los puntos asociados a cada uno de ellas.

* Utilizar este jupyter notebook para probar tu código. Contiene un ejemplo sencillo de cómo tengo organizado mi script, pero puedes modificarlo como gustes, o incluso borrar todo y empezar de cero. Recuerda que necesitamos este jupyter notebook, ya que tendrá tu desarrollo paso a paso y el cómo pensaste los ejercicios.

* Durante el proceso, debes materializar al menos dos tablas en la db, referentes a:
    * Datos del Mercado como los recibes.
    * Datos generados a partir de tus transformaciones y tu estrategia (si creas alguna). No debes repetir los datos del mercado del punto anterior.

* Crear un archivo producer.py que contendrá todo el código necesario para el producer.

* Repetir el punto anterior para el consumer.

### Consejos y tips.

* Recuerda que el objetivo es separar los servicios, por lo que es esperado que trabajes con al menos 2 diferentes archivos, pero puedes trabajar incluso con más de dos archivos, si por ejemplo deseas más organización o si es complicado orgnizar las ideas y las conexiones entre ellas en un solo archivo. A veces ayuda separarlo todo y pensar la lógica de manera granular.

* También recuerda que solo los mensajes dentro de una partición están ordenados, NO se ordenan automáticamente si se envían a diferentes particiones, por lo que si haces esto, debes tener una forma de organizarlos y procesarlos posterior a recibirlos en los consumer.

* Enfocate en las funciones objetivo, después de ello, pasa a la segunda sección, de esa forma aseguras los puntos fáciles.

### TBD

Como ya lo mencioné, estamos a contra reloj creando el material para ustedes, por lo que los siguientes puntos están pendientes de entregarles.

* Es probable que añada algunas funciones más en las funciones objetivo, estén al pendiente durante el fin de semana.

* Las funciones extras: Agregué solo una de momento, pero hace falta agregar más.

* Incorporar servicios:
    - Alcanza el tiempo?
    - Cuáles?

* Los puntos asignados a cada función.

* Lo más probable es que KSQL no lo alcancemos a ver, pero aún hay esperanza.

In [ ]:
from IPython.display import clear_output # You can use this to clean your output in every loop.
from json import loads

from datetime import datetime as dt
import consumer_functions as cf
import project_functions as proj_f

### DB Connection Variables.
###### Recuerda NO subir tus credenciales a github: siempre puedes crear un JSON, mantener tus credenciales ahí e ingresarlo en tu .gitignore.

json_creds = loads('/path/to/json/credentials')
user = json_creds['']
password = json_creds['']
host = json_creds['']
db = json_creds['']
table_name = 'BTCFDUSD'

### Topic name.
dw_mk_topic='dw.market.data'

# Connection to our database.
engine = cf.db_engine(user,password,host,db)

# Consumer creation and TopicPartition usage.
dw_mk_consumer = cf.consumer(dw_mk_topic)

# Loop over the consumer for the latest message they received.
for dw_mk_msg in dw_mk_consumer:
    
    # Cleaning the output on the cells of .ipynb files.
    # clear_output(wait=True) 
    
    if dw_mk_msg.value:
        # Handling the message value into readable format and creating a dataframe from it.
        cf.handling_message(dw_mk_msg)
        cf.db_materialization(table_name,engine)

In [ ]:
from datetime import datetime as dt
import producer_functions as prod_f
import project_functions as proj_f

current_time = dt.now()

### get_market_data() & processing_market_data().
type_market_data = ''
symbol = ''
interval = ''
rows = ''

prod_f.create_producer()

i = 0
while True:
    
    ##################################################################################################################################
    # Descarga y limpieza de datos de X días atrás a hoy.
    ##################################################################################################################################       
    recent_data, market_data_msg, received_time = prod_f.processing_market_data(type_market_data,symbol,interval,rows)
    
    if current_time != recent_data.iloc[-1,0]:
        
        recent_data = prod_f.transformations(recent_data)
        recent_data = proj_f.strategy(recent_data)